# Breif SATPY overview w/ MODIS - VIIRS - MSI data

Data Used: located in shared folder

In [99]:
# Only packages required for this notebook but many more may be useful
import numpy as np
import matplotlib.pyplot as plt
from pyart.default_config import corrected_reflectivity
from satpy import Scene, find_files_and_readers, DataQuery
from datetime import datetime
import os
from pathlib import Path
from collections import defaultdict

from satpy.demo.viirs_sdr import SVDNB_FILES

from satellite_data_processor import *

In [100]:
def plot_difference_histogram(
    corrected,
    uncorrected,
    *,
    percent=False,
    bins=100,
    range_percentile=99,
    title=None,
    ax=None
):
    """
    Plot a histogram of band differences (corrected - uncorrected).

    Parameters
    ----------
    corrected : ndarray
        Corrected band data
    uncorrected : ndarray
        Uncorrected band data
    percent : bool, optional
        If True, plot percent difference: 100 * (c - u) / u
    bins : int, optional
        Number of histogram bins
    range_percentile : float, optional
        Percentile used to limit histogram range (symmetric)
    title : str, optional
        Plot title
    ax : matplotlib axis, optional
        Existing axis to plot on

    Returns
    -------
    stats : dict
        Dictionary of summary statistics
    """

    # Compute difference
    if percent:
        with np.errstate(divide="ignore", invalid="ignore"):
            diff = 100.0 * (corrected - uncorrected) / uncorrected
        xlabel = "Percent difference (%)"
    else:
        diff = np.array(corrected - uncorrected)
        xlabel = "Difference (corrected − uncorrected)"

    # Flatten and remove NaNs/Infs
    diff = diff.ravel()
    diff = diff[np.isfinite(diff)]

    # Robust range using percentiles
    p = np.nanpercentile(np.abs(diff), range_percentile)
    hist_range = (-p, p)

    # Create axis if needed
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 4))

    # Plot histogram
    ax.hist(diff, bins=bins, range=hist_range, histtype="stepfilled", alpha=0.7)

    # Statistics
    mean = np.nanmean(diff)
    median = np.nanmedian(diff)
    std = np.nanstd(diff)
    rmse = np.sqrt(np.nanmean(diff**2))

    # Overlay statistics
    ax.axvline(mean, linestyle="--", linewidth=2, label=f"Mean = {mean:.3g}")
    ax.axvline(median, linestyle=":", linewidth=2, label=f"Median = {median:.3g}")

    # Labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Pixel count")
    ax.legend()

    if title is not None:
        ax.set_title(title)

    stats = {
        "mean": mean,
        "median": median,
        "std": std,
        "rmse": rmse,
        "n_pixels": diff.size,
    }

    return stats


In [101]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module="dask"
)


In [102]:
# Create symlink to fix the naming issue ... you may need to change this depending on your system
pyspectral_dir = Path.home() / 'Library/Application Support/pyspectral'
source = pyspectral_dir / 'rsr_modis_EOS-Aqua.h5'
target = pyspectral_dir / 'rsr_modis_Aqua.h5'

if source.exists() and not target.exists():
    os.symlink(source, target)
    print(f"Created symlink: {target} -> {source}")
else:
    print(f"Source exists: {source.exists()}, Target exists: {target.exists()}")

Source exists: True, Target exists: True


In [103]:
# CHANGE THIS ---------------------------------- (if running notebook in folder then Path('.') should work)
DATA_DIR = str(Path('~/Downloads').expanduser())

####  --- MODIS ---
Link below contains available composites as well as modifiers for corrections

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/modis.yaml


#### A Note On Corrections

* See https://satpy.readthedocs.io/en/latest/api/satpy.modifiers.geometry.html for geometric corrections

* See https://pyspectral.readthedocs.io/en/latest/rayleigh_correction.html for radiometric/atmospheric correction

This cell uses Satpy’s `find_files_and_readers` function to identify all files in the specified directory that can be read by the chosen reader.

**Note:** This function returns **all files within the requested time bounds**. If your directory contains MODIS data from both **Aqua and Terra**, files from *both platforms* will be returned, so take care to filter appropriately if you only want one sensor.

The resulting list of files is passed to the `Scene` class, which can ingest both radiance/reflectance files and their corresponding geolocation files automatically.

While the `Scene` class is very powerful, it can become unwieldy when handling multiple, temporally separated datasets. For this reason, it is recommended to create a **separate `Scene` instance for each analysis time**.

For example, rather than loading data for
`event_1 = (2023, 1, 3, 13, 0)` and
`event_2 = (2025, 1, 1, 3, 0)`
into a single `Scene`, you should create one `Scene` per event. This keeps each scene temporally consistent and avoids unintended file mixing.

Additionally, it is also recommended to create a **separate `Scene` instance for applying corrections to the data**. This is to help avoid issues with the load/unload functionality and ensure the correct procedures are being followed.


In [94]:
# note that I am only using one granule here...
myfiles = find_files_and_readers(base_dir=DATA_DIR,
                                 sensor="modis",
                                 start_time=datetime(2026, 1, 1, 12, 30),
                                 end_time=datetime(2026, 1, 1, 12, 30),
                                 reader='modis_l1b',)

# both uncorrected and corrected scenes are initialized
uncorrected_scn = Scene(filenames=myfiles)
corrected_scn = Scene(filenames=myfiles)

In [98]:
corrected_scn.

TypeError: Scene.__contains__() missing 1 required positional argument: 'name'

In [33]:
# The Scene can tell you what datasets are availabe to load based on the files that you have initialized the class with
# note that I have provided the Scene with the geolocation file.
print(uncorrected_scn.available_dataset_names())
print()
print(corrected_scn.available_dataset_names())

['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9', 'height', 'landsea_mask', 'latitude', 'longitude', 'range', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'waterpresent']

['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9', 'height', 'landsea_mask', 'latitude', 'longitude', 'range', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'waterpresent']


In [34]:
corrected_scn.load(['height','range','satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle'])
corrected_scn.load(terra_ir)
corrected_scn.load(terra_swir_nir, modifiers=('sunz_corrected',))
corrected_scn.load(terra_visible, modifiers=('sunz_corrected', 'rayleigh_corrected'))
corrected_scn.load(['true_color'])
corrected_scn.coarsest_area()

The following datasets were not created and may require resampling to be generated: DataID(name='16', wavelength=WavelengthRange(min=0.862, central=0.8695, max=0.877, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='5', wavelength=WavelengthRange(min=1.23, central=1.24, max=1.25, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='17', wavelength=WavelengthRange(min=0.89, central=0.905, max=0.92, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='19', wavelength=WavelengthRange(min=0.915, central=0.94, max=0.965, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='15', wavelength=WavelengthRange(min=0.743, central=0.748, max=0.753, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='18', wavelength=WavelengthRange(min=0.931, central=0.936, max=0.941, unit='µm'), resolution=1000, calibration

Shape: (2030, 1354)
Lons: <xarray.DataArray 'Longitude-14f53a0ac836d4416180581c58534db7' (y: 2030, x: 1354)> Size: 11MB
dask.array<scale_and_mask, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x
Attributes: (12/15)
    units:                degree
    valid_range:          [-180.0, 180.0]
    standard_name:        longitude
    platform_name:        Aqua
    sensor:               modis
    rows_per_scan:        10
    ...                   ...
    name:                 longitude
    resolution:           1000
    modifiers:            ()
    _satpy_id:            DataID(name='longitude', resolution=1000, modifiers...
    long_name:            longitude
    ancillary_variables:  []
Lats: <xarray.DataArray 'Latitude-3b2d5e94c4411a91c228866856b49c90' (y: 2030, x: 1354)> Size: 11MB
dask.array<scale_and_mask, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x
Attributes: (12/15)
    units:                degree
    valid_range:          [-90.0, 90.0]
    standard_name:        latitude
    platform_name:        Aqua
    sensor:               modis
    rows_per_scan:        10
    ...                   ...
    name:                 latitude
    resolution:           1000
    modifiers:            ()
    _satpy_id:            DataID(name='latitude', resolution=1000, modifiers=())
    long_name:            latitude
    ancillary_variables:  []

In [38]:
print(corrected_scn.available_composite_names())

Too many possible datasets to load for DataQuery(wavelength=0.67)


['24h_microphysics', 'airmass', 'ash', 'day_essl_colorized_low_level_moisture', 'day_essl_low_level_moisture', 'day_microphysics', 'dust', 'essl_colorized_low_level_moisture', 'essl_low_level_moisture', 'fog', 'green_snow', 'ir108_3d', 'ir_cloud_day', 'natural_color', 'natural_color_raw', 'natural_with_night_fog', 'night_fog', 'ocean_color', 'overview', 'snow', 'true_color', 'true_color_crefl', 'true_color_thin', 'true_color_uncorrected']


In [39]:
corrected_scn.load(['true_color'])
corrected_scn['true_color']

The following datasets were not created and may require resampling to be generated: DataID(name='1', wavelength=WavelengthRange(min=0.62, central=0.645, max=0.67, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected', 'rayleigh_corrected')), DataID(name='5', wavelength=WavelengthRange(min=1.23, central=1.24, max=1.25, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='19', wavelength=WavelengthRange(min=0.915, central=0.94, max=0.965, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='14hi', wavelength=WavelengthRange(min=0.673, central=0.678, max=0.683, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected', 'rayleigh_corrected')), DataID(name='15', wavelength=WavelengthRange(min=0.743, central=0.748, max=0.753, unit='µm'), resolution=1000, calibration=<1>, modifiers=('sunz_corrected',)), DataID(name='13lo', wavelength=WavelengthRange(min=0.662, central=0.667, max=0.6

KeyError: "No dataset matching 'DataQuery(name='true_color')' found"

In [8]:
resampled = corrected_scn.resample(corrected_scn.coarsest_area(), resampler='native')

Could not get the reflectance correction using band name: 13hi
Will try use the wavelength, however, this may be ambiguous!
A wavelength is provided instead of band name - disregard the relative spectral responses and assume it is the effective wavelength: 0.667000 (micro meter)
Could not get the reflectance correction using band name: 13lo
Will try use the wavelength, however, this may be ambiguous!
A wavelength is provided instead of band name - disregard the relative spectral responses and assume it is the effective wavelength: 0.667000 (micro meter)
Could not get the reflectance correction using band name: 14hi
Will try use the wavelength, however, this may be ambiguous!
A wavelength is provided instead of band name - disregard the relative spectral responses and assume it is the effective wavelength: 0.678000 (micro meter)
Could not get the reflectance correction using band name: 14lo
Will try use the wavelength, however, this may be ambiguous!
A wavelength is provided instead of 

In [42]:
print(resampled['5'])

<xarray.DataArray 'getitem-0305cb338a1bbdc2c2f4466282fa9cad' (y: 2030, x: 1354)> Size: 11MB
dask.array<_sunzen_corr_cos_ndarray, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>
Coordinates:
    crs      object 8B +proj=longlat +ellps=WGS84 +type=crs
Dimensions without coordinates: y, x
Attributes: (12/18)
    name:                 5
    resolution:           1000
    calibration:          reflectance
    coordinates:          ('longitude', 'latitude')
    wavelength:           1.24 µm (1.23-1.25 µm)
    file_type:            hdf_eos_data_1000m
    ...                   ...
    start_time:           2026-01-01 12:30:00
    end_time:             2026-01-01 12:35:00
    reader:               modis_l1b
    area:                 Shape: (2030, 1354)\nLons: <xarray.DataArray 'Longi...
    _satpy_id:            DataID(name='5', wavelength=WavelengthRange(min=1.2...
    ancillary_variables:  []


In [23]:
resampled['2'].area.get_lonlats()

(dask.array<scale_and_mask, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>,
 dask.array<scale_and_mask, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>)

In [11]:
resampled['longitude'], resampled['latitude'] = resampled.coarsest_area().get_lonlats()

ValueError: Key must be a DataID when value is not an xarray DataArray or dict

In [11]:
xr = (resampled.to_xarray(include_lonlats=True))
xr

<xarray.Dataset> Size: 528MB
Dimensions:                  (y: 2030, x: 1354, bands: 3)
Coordinates:
    longitude                (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    latitude                 (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
  * bands                    (bands) <U1 12B 'R' 'G' 'B'
Dimensions without coordinates: y, x
Data variables: (12/44)
    CHANNEL_1                (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    CHANNEL_10               (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    CHANNEL_11               (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    CHANNEL_12               (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    CHANNEL_13hi             (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    CHANNEL_13lo             (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    ...                       ...
    range                    (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    satellite_azimuth_angle  (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    satellite_zenith_angle   (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    solar_azimuth_angle      (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    solar_zenith_angle       (y, x) float32 11MB dask.array<chunksize=(1540, 1354), meta=np.ndarray>
    true_color               (bands, y, x) float32 33MB dask.array<chunksize=(1, 1540, 1354), meta=np.ndarray>
Attributes:
    history:      Created by pytroll/satpy on 2026-02-17 02:30:40.844992+00:00
    Conventions:  CF-1.7

In [43]:
xr.isel(y=434, x=434)[('CHANNEL_1')].values

array(1.2427251, dtype=float32)

In [45]:
xr.isel(y=434, x=434)[('CHANNEL_4')].values


array(2.2194047, dtype=float32)

In [30]:
xr.true_color[:,434, 434]

<xarray.DataArray 'true_color' (bands: 3)> Size: 12B
dask.array<getitem, shape=(3,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    longitude  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    latitude   float32 4B dask.array<chunksize=(), meta=np.ndarray>
  * bands      (bands) <U1 12B 'R' 'G' 'B'
Attributes:
    end_time:                2026-01-01 12:35:00
    file_type:               hdf_eos_data_1000m
    mode:                    RGB
    optional_prerequisites:  []
    platform_name:           Aqua
    prerequisites:           ["DataQuery(name='1', modifiers=('sunz_corrected...
    reader:                  modis_l1b
    resolution:              1000
    rows_per_scan:           10
    sensor:                  modis
    standard_name:           true_color
    start_time:              2026-01-01 12:30:00

In [23]:
xr.true_color[:,434,434].values

array([1.2427251, 2.2194047, 4.9701366], dtype=float32)

In [ ]:
uncorrected_scn.load(['1', 'latitude','longitude', 'solar_zenith_angle'])
# corrected_scn.load(['latitude','longitude', 'solar_zenith_angle'])
corrected_scn.load(['1'], modifiers=['sunz_corrected'])

tc_scene = Scene(filenames=myfiles)



In [ ]:
tc_scene.load(['true_color'])


In [ ]:
tc_rs = tc_scene.resample(tc_scene.coarsest_area(),resampler='native')


In [ ]:
array = np.transpose(tc_rs['true_color'].values, (1,2,0))

In [ ]:
np.nanmax(array)

In [ ]:
fig, ax = plt.subplots()
ax.imshow((array-np.nanmin(array)) / (100 - np.nanmin(array)))

In [ ]:
resampled_cor = corrected_scn.resample(corrected_scn.coarsest_area(), resampler='native')
resampled_uncor = uncorrected_scn.resample(uncorrected_scn.coarsest_area(), resampler='native')


In [ ]:
print(uncorrected_scn['1'].attrs['modifiers'])
print(resampled_cor['1'].attrs['modifiers'])

In [ ]:
print(resampled_uncor['1'].shape)
print(resampled_cor['1'].shape)

In [ ]:
resampled_uncor['1'].plot(vmin=0, vmax=100)

In [ ]:
resampled_cor['1'].plot(vmin=0, vmax=100)

In [ ]:
resampled_cor['1'].plot()

#### Loading Data

To access data from the `Scene` class the `.load` method is used to pull the data from the provided datasets. These datasets are built on xarray and dask so they are lazy loaded and need to be computed for any of the arrays to produce values. The `Scene` class holds all of these datasets and they can be acessed via slicing by key as shown later.

A safe and effective process for using the `.load` method is outlined below:

If no corrections:
* you can load all the data at once

If doing corrections:
* load data that does not require correction
* load data that requires minimal correction
* resample data
* load data that requires maximal correction
* resample again to same resampling method as above



In [ ]:
# uncorrected
uncorrected_scn.load(uncorrected_scn.available_dataset_names()[:-1]) # loads in all but the waterpresent (has issues)
print(uncorrected_scn)

In [ ]:
# corrected

# load in the geolocation datasets first
corrected_scn.load(['height', 'landsea_mask',
                    'latitude', 'longitude',
                    'range', 'satellite_azimuth_angle',
                    'satellite_zenith_angle', 'solar_azimuth_angle',
                    'solar_zenith_angle'])

In [ ]:
print(corrected_scn.available_dataset_names())

In [ ]:
# now that geolocaiton is loaded in proceed to adding bands with minimal correction ... in this case solar/satellite zenith corrections.
# I will do this for all but bands 1, 3, and 4 where I will apply additional correction in the next step

# corrections are applied by specifying the type of correction with the modifiers option as shown below
# note that the available modifiers can be found in the github doc link provided

corrected_scn.load(['10', '11', '12', '13hi', '13lo', '14hi',
                    '14lo', '15', '16', '17', '18', '19',
                    '2', '20', '21', '22', '23', '24',
                    '25', '26', '27', '28', '29', '30',
                    '31', '32', '33', '34', '35', '36',
                    '5', '6', '7', '8', '9'],
                   modifiers=['effective_solar_pathlength_corrected'])

# running with the modifiers may yield a warning that the dataset has not been loaded and resampling may be needed.
# we will discuss resampling in the last step but proceed with loading the next set of data

In [ ]:
corrected_scn.load(['1', '3', '4'], modifiers=['effective_solar_pathlength_corrected','rayleigh_corrected'])

#### Resampling the data

The `.resample` method is extremly important and it allows us to create the datasets that were not generated above as well as generate composites.
Different resamplings can be done and I suggest you take a look at the documentation but for the purposes of the project the data should
use the 'native' resampler to apply the correct satellite geometry data and projection to the datasets. By using the 'native' resampler
we are basically getting back the default pixels that the modis scanner is sensing while allowing for satpy to know that all the data is
on the same grid.

In [ ]:
# resample

# IMPORTANT --- SAVE THE NEW SCENE!!!!  just calling corrected_scn.resample() will not do anything you have to save as a variable!
modis_corrected_scn = corrected_scn.resample(resampler='native')

In [ ]:
# lets check that the datasets are loaded and have the correct modifiers
print(corrected_scn['10'].attrs['modifiers'])
print(modis_corrected_scn['10'].attrs['modifiers'])
print()
print(corrected_scn['1'].attrs['modifiers'])
print(modis_corrected_scn['1'].attrs['modifiers'])

In [ ]:
# lets take a look at the available default composites... These can also be found at the github link
# note that new composites can be made / generate ... see satpy documentation for more.
print(modis_corrected_scn.available_composite_names())

In [ ]:
# there is an uncorrected version of truecolor
uncorrected_scn.load(['true_color_uncorrected'])
# corrected more useful version ...
corrected_scn.load(['true_color'])

# note that corrected_scn is being used again... once you resample you dont have access to the composites... So make sure to load all the data you want included before resampling

# note that you do not have to load these with corrections as they will alread call the necessary corrections for you and handle it in the back end

# dont forget to resample...
resampled_uncorr = uncorrected_scn.resample(resampler='native')
resampled_corr = corrected_scn.resample(resampler='native')



In [ ]:
# modifiers can be checked for using the .prerequisites method for the composites. This shows each band used and its correction
print(resampled_uncorr['true_color_uncorrected'].prerequisites)
print()
print(resampled_corr['true_color'].prerequisites)

Note that resampled_uncorr now has sunz_corected as a modifier...

In [ ]:
# generate boolean array to see if band 1 has different values for corrected and uncorrected bands. We should expect they are mostly different...
_band1_boolean = resampled_corr['1'] == resampled_uncorr['1']
np_boolean_results = np.array(_band1_boolean)
numpy_true_count = np.sum(np_boolean_results)
numpy_false_count = np.sum(~np_boolean_results)

print('True count:', numpy_true_count)
print('False count:', numpy_false_count)

In [ ]:
resampled_uncorr['31'].attrs

In [ ]:
corrected_scn.available_dataset_ids()

In [ ]:
test = corrected_scn.load(['1'])

### Test Out Corrections

In [ ]:
# lets take a look at a histogram for difference values and their frequency of occurrence + some statistics
plot_difference_histogram(corrected=resampled_corr['1'], uncorrected=resampled_uncorr['1'])

In [ ]:
# raw uncorrected plot
%matplotlib inline

fig = plt.figure()
resampled_uncorr['1'].plot()
plt.show()


In [ ]:
uncorrected_scn['1'].attrs['area']

In [ ]:
resampled_corr['1'].plot(vmin=0)


In [ ]:
resampled_uncorr['31'].plot()

In [ ]:
print(corrected_scn.available_dataset_names())
print()
print(corrected_scn['solar_zenith_angle'])

In [ ]:
resampled_corr['31'].plot()

In [ ]:
resampled_corr['true_color']s

In [ ]:
fig, ax = plt.subplots()
ax.imshow(resampled_corr['true_color'])

In [ ]:
image = np.asarray(resampled_corr["true_color"]).transpose(1,2,0)
image = np.nan_to_num(image)
image = np.interp(image, (np.percentile(image,1), np.percentile(image,99)), (0, 1))
crs = resampled_corr["true_color"].attrs["area"].to_cartopy_crs()


In [ ]:

import matplotlib.pyplot as plt
from satpy.writers import get_enhanced_image

plt.figure()
img = get_enhanced_image(resampled_corr['true_color'])
# get DataArray out of `XRImage` object
img_data = img.data
img_data.plot.imshow(rgb='bands', vmin=0, vmax=1)

In [ ]:
# Initiate a matplotlib figure
fig = plt.figure(figsize=(16, 10))

ax = fig.add_subplot(1, 1, 1, projection=crs)

# Specify coastlines
ax.coastlines(resolution="10m", color="white")

# Specify a grid
gl = ax.gridlines(draw_labels=True, linestyle='--', xlocs=range(-140,-110,5), ylocs=range(20,50,5))
gl.top_labels=False
gl.right_labels=False
gl.xformatter=LONGITUDE_FORMATTER
gl.yformatter=LATITUDE_FORMATTER
gl.xlabel_style={'size':14}
gl.ylabel_style={'size':14}

# Plotting function
ax.imshow(image, transform=crs, extent=crs.bounds, origin="upper")

# Set plot title
plt.title("Natural color composite of California, recorded by MODIS at " + scn['1'].attrs['start_time'].strftime("%Y-%m-%d %H:%M"), fontsize=20, pad=20.0)

# Show the plot
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(resampled_corr['1'])

In [ ]:
# raw corrected plot
fig, ax = plt.subplots()
ax.imshow(resampled_corr['1'])

In [ ]:
# diverging difference plot...
fig, ax = plt.subplots()
ax.imshow(resampled_corr['1']-resampled_uncorr['1'],
          cmap='bwr', vmin=-30, vmax=30)

### --- VIIRS ---

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/viirs.yaml

In [104]:
viirs_files = find_files_and_readers(base_dir=DATA_DIR,
                                     start_time=datetime(2026, 1, 1, 12, 00),
                                     end_time=datetime(2026, 1, 1, 12, 00),
                                     reader='viirs_l1b',)
print(viirs_files)
print()

viirs_uncorrected_scn = Scene(filenames=viirs_files)
viirs_corrected_scn = Scene(filenames=viirs_files)

print(viirs_uncorrected_scn.all_dataset_names())

{'viirs_l1b': ['/Users/cwelch/Downloads/VJ103IMG.A2026001.1200.021.2026001183040.nc', '/Users/cwelch/Downloads/VJ103MOD.A2026001.1200.021.2026001183040.nc', '/Users/cwelch/Downloads/VJ102IMG.A2026001.1200.021.2026001195531.nc', '/Users/cwelch/Downloads/VJ102MOD.A2026001.1200.021.2026001195531.nc']}

['DNB', 'I01', 'I02', 'I03', 'I04', 'I05', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'dnb_lat', 'dnb_lon', 'dnb_lunar_azimuth_angle', 'dnb_lunar_zenith_angle', 'dnb_moon_illumination_fraction', 'dnb_satellite_azimuth_angle', 'dnb_satellite_zenith_angle', 'dnb_solar_azimuth_angle', 'dnb_solar_zenith_angle', 'i_lat', 'i_lon', 'm_lat', 'm_lon', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle']


In [56]:
viirs_data_load_list = viirs_uncorrected_scn.all_dataset_names()[1:22] + viirs_uncorrected_scn.all_dataset_names()[-8:]

viirs_uncorrected_scn.load(viirs_data_load_list)
# ignore the xarray warnings

In [57]:
viirs_corrected_scn.load([
    # 'i_lat', 'i_lon', 'm_lat', 'm_lon',
    'satellite_azimuth_angle', 'satellite_zenith_angle',
    'solar_azimuth_angle', 'solar_zenith_angle'
]
)

viirs_corrected_scn.load(['M01', 'M02','M04', 'M05', 'M06', 'M07', 'M08',
                          'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16'],
                         modifiers=['sunz_corrected'])

# note that VIIRS has a separate correction for the I-bands
viirs_corrected_scn.load(['I01', 'I02', 'I03', 'I04', 'I05'],
                         modifiers=['sunz_corrected_iband'])

viirs_corrected_scn.load(['M03'], modifiers=['sunz_corrected', 'rayleigh_corrected'])

resampled_viirs = viirs_corrected_scn.resample(viirs_corrected_scn.coarsest_area(), resampler='native')

print(viirs_corrected_scn['M01'].attrs['modifiers'])
print()
print(viirs_corrected_scn['I03'].attrs['modifiers'])
print()
print(viirs_corrected_scn['M03'].attrs['modifiers'])

('sunz_corrected',)

('sunz_corrected_iband',)

('sunz_corrected', 'rayleigh_corrected')


In [61]:
viirs_corrected_scn.coarsest_area()

Shape: (3232, 3200)
Lons: <xarray.DataArray 'longitude' (y: 3232, x: 3200)> Size: 41MB
dask.array<where, shape=(3232, 3200), dtype=float32, chunksize=(3232, 3200), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x
Attributes: (12/25)
    start_orbit:          42082
    end_orbit:            42082
    orbital_parameters:   {'start_direction': 'Ascending', 'end_direction': '...
    long_name:            Longitudes of pixel locations
    units:                degrees
    _FillValue:           -999.9
    ...                   ...
    day_night:            Day
    start_time:           2026-01-01 12:00:00
    end_time:             2026-01-01 12:06:00
    reader:               viirs_l1b
    _satpy_id:            DataID(name='m_lon', resolution=742, modifiers=())
    ancillary_variables:  []
Lats: <xarray.DataArray 'latitude' (y: 3232, x: 3200)> Size: 41MB
dask.array<where, shape=(3232, 3200), dtype=float32, chunksize=(3232, 3200), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x
Attributes: (12/25)
    start_orbit:          42082
    end_orbit:            42082
    orbital_parameters:   {'start_direction': 'Ascending', 'end_direction': '...
    long_name:            Latitudes of pixel locations
    units:                degrees
    _FillValue:           -999.9
    ...                   ...
    day_night:            Day
    start_time:           2026-01-01 12:00:00
    end_time:             2026-01-01 12:06:00
    reader:               viirs_l1b
    _satpy_id:            DataID(name='m_lat', resolution=742, modifiers=())
    ancillary_variables:  []

In [63]:
dir(resampled_viirs.coarsest_area())

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_compute_generic_parameters',
 '_compute_omerc_parameters',
 '_compute_uniform_shape',
 '_corner_is_clockwise',
 '_do_transform',
 '_extract_lonlat_subarrays',
 '_filter_sides_nans',
 '_get_bbox_slices',
 '_get_geographic_sides',
 '_get_geostationary_boundary_sides',
 '_get_sides',
 '_repr_html_',
 '_reverse_boundaries',
 'aggregate',
 'append',
 'boundary',
 'cartesian_coords',
 'compute_bb_proj_params',
 'compute_optimal_bb_area',
 'concatenate',
 'copy',
 'corners',
 'crs',
 'dtype',
 'geocentric_resolution',
 'get_area',
 'get_area_extent_for_subset'

In [71]:
print(resampled_viirs.coarsest_area().lons.attrs['resolution'])

742


In [75]:
if 'true_color'  in viirs_corrected_scn.available_composite_names():
    print('haza')

haza


In [10]:
resampled_viirs.to_xarray(include_lonlats=True)

<xarray.Dataset> Size: 1GB
Dimensions:                  (y: 3232, x: 3200)
Coordinates:
    longitude                (y, x) float32 41MB 80.33 80.31 ... 8.187 8.161
    latitude                 (y, x) float32 41MB -63.33 -63.34 ... -55.14 -55.14
Dimensions without coordinates: y, x
Data variables: (12/25)
    I01                      (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    I02                      (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    I03                      (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    I04                      (y, x) float32 41MB dask.array<chunksize=(1024, 2048), meta=np.ndarray>
    I05                      (y, x) float32 41MB dask.array<chunksize=(1024, 2048), meta=np.ndarray>
    M01                      (y, x) float32 41MB dask.array<chunksize=(3232, 3200), meta=np.ndarray>
    ...                       ...
    M15                      (y, x) float32 41MB dask.array<chunksize=(3232, 3200), meta=np.ndarray>
    M16                      (y, x) float32 41MB dask.array<chunksize=(3232, 3200), meta=np.ndarray>
    satellite_azimuth_angle  (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    satellite_zenith_angle   (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    solar_azimuth_angle      (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    solar_zenith_angle       (y, x) float32 41MB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
Attributes:
    history:      Created by pytroll/satpy on 2026-02-17 02:04:03.437660+00:00
    Conventions:  CF-1.7

### --- SENTINEL-2 ---

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/sen2_msi.yaml

/Users/cwelch/Downloads/sars_p1_data


In [42]:
msi_files = find_files_and_readers(base_dir=DATA_DIR,
                                     start_time=datetime(2026, 1, 1, 12, 00),
                                     end_time=datetime(2026, 1, 1, 12, 00),
                                     reader='msi_safe',)
print(msi_files)
msi_uncorrected_scn = Scene(reader='msi_safe', filenames=msi_files)
msi_corrected_scn = Scene(filenames=msi_files)
print()
print(msi_uncorrected_scn.all_dataset_names())

{'msi_safe': ['/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/MTD_TL.xml', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/MTD_MSIL1C.xml', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B02.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B07.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B09.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B08.jp2', '/Users/cwelch/Downloads/S2

In [7]:
len(msi_files['msi_safe'])

16

In [8]:
msi_files['msi_safe']

['/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/MTD_TL.xml',
 '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/MTD_MSIL1C.xml',
 '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B02.jp2',
 '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B07.jp2',
 '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B09.jp2',
 '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B08.jp2',
 '/Users/cwelch/Downloads/S2A_MSIL1

In [9]:
print(msi_corrected_scn.available_composite_names())

['cloud_phase', 'cloud_phase_raw', 'dataspace_swir', 'dataspace_swir_uncorr', 'day_essl_colorized_low_level_moisture', 'day_essl_low_level_moisture', 'essl_colorized_low_level_moisture', 'essl_low_level_moisture', 'false_color', 'natural_color', 'ndmi', 'ndsi', 'ndsi_with_true_color', 'ndvi', 'ndwi', 'true_color', 'true_color_antarctic', 'true_color_continental_average', 'true_color_continental_clean', 'true_color_continental_polluted', 'true_color_desert', 'true_color_marine_clean', 'true_color_marine_polluted', 'true_color_marine_tropical', 'true_color_raw', 'true_color_rural', 'true_color_uncorr', 'true_color_urban', 'urban_color']


In [44]:
msi_corrected_scn.load(['B02'])
print(msi_corrected_scn['B02'])

<xarray.DataArray 'band_data' (y: 10980, x: 10980)> Size: 482MB
dask.array<mul, shape=(10980, 10980), dtype=float32, chunksize=(4096, 4096), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 88kB 8e+05 8e+05 8e+05 ... 9.098e+05 9.098e+05
  * y            (y) float64 88kB 7.6e+06 7.6e+06 7.6e+06 ... 7.49e+06 7.49e+06
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["WGS 84 / UTM zone 23S",BASEGEOGCRS["WGS 8...
Attributes: (12/16)
    name:                 B02
    sensor:               msi
    wavelength:           0.49 µm (0.44-0.54 µm)
    resolution:           10
    calibration:          reflectance
    file_type:            l1c_safe_granule
    ...                   ...
    start_time:           2026-01-01 12:58:41.058082
    end_time:             2026-01-01 12:58:41.058082
    reader:               msi_safe
    area:                 Area ID: 23KRR\nDescription: On-the-fly area\nProje...
    _satpy_id:            DataID(name

In [105]:
msi_corrected_scn.coarsest_area()

Area ID: 23KRR
Description: On-the-fly area
Projection ID: 23KRR
Projection: {'datum': 'WGS84', 'no_defs': 'None', 'proj': 'utm', 'south': 'None', 'type': 'crs', 'units': 'm', 'zone': '23'}
Number of columns: 1830
Number of rows: 1830
Area extent: (799980.0, 7490200.0, 909780.0, 7600000.0)

In [77]:
bandz = [
    'true_color',
    'natural_color',
    'false_color',
    'urban_color',
    'ndvi',
    'aerosol_optical_thickness',
]

In [88]:
bandz = [
    'true_color',
    'natural_color',
    'false_color',
    'urban_color',
    'ndvi',
    # 'aerosol_optical_thickness',
]
from pyresample.resampler import AreaDefinition
if isinstance(msi_corrected_scn.coarsest_area(), AreaDefinition) and all(band in msi_corrected_scn.available_composite_names() for band in bandz):
    print(bandz)

['true_color', 'natural_color', 'false_color', 'urban_color', 'ndvi']


In [87]:
msi_corrected_scn.available_composite_names()

['cloud_phase',
 'cloud_phase_raw',
 'dataspace_swir',
 'dataspace_swir_uncorr',
 'day_essl_colorized_low_level_moisture',
 'day_essl_low_level_moisture',
 'essl_colorized_low_level_moisture',
 'essl_low_level_moisture',
 'false_color',
 'natural_color',
 'ndmi',
 'ndsi',
 'ndsi_with_true_color',
 'ndvi',
 'ndwi',
 'true_color',
 'true_color_antarctic',
 'true_color_continental_average',
 'true_color_continental_clean',
 'true_color_continental_polluted',
 'true_color_desert',
 'true_color_marine_clean',
 'true_color_marine_polluted',
 'true_color_marine_tropical',
 'true_color_raw',
 'true_color_rural',
 'true_color_uncorr',
 'true_color_urban',
 'urban_color']

In [85]:
all(band in msi_corrected_scn.available_composite_names() for band in bandz)

False

In [86]:
list1 = ['a', 'b', 'c']
list2 = ['a', 'b', 'c', 'd', 'e']

if all(item in list2 for item in list1):
    print("All elements of list1 are in list2")
else:
    print("Some elements of list1 are missing in list2")

All elements of list1 are in list2


In [52]:
res = msi_corrected_scn.coarsest_area().pixel_size_x

60.0

In [54]:
msi_corrected_scn.load(['true_color'], resolution=msi_corrected_scn.coarsest_area().pixel_size_x)

The following datasets were not created and may require resampling to be generated: DataID(name='true_color')


In [46]:
resamp = msi_corrected_scn.resample(msi_corrected_scn.coarsest_area(), resampler='native')

In [48]:
resamp['true_color']

<xarray.DataArray 'where-b645b04310d9a08c65d4330ae6bfa704' (bands: 3, y: 1830,
                                                            x: 1830)> Size: 80MB
dask.array<where, shape=(3, 1830, 1830), dtype=float64, chunksize=(1, 1394, 1394), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["WGS 84 / UTM zone 23S",BASEGEOGCRS["WGS 8...
  * y            (y) float64 15kB 7.6e+06 7.6e+06 7.6e+06 ... 7.49e+06 7.49e+06
  * x            (x) float64 15kB 8e+05 8.001e+05 ... 9.097e+05 9.098e+05
  * bands        (bands) <U1 12B 'R' 'G' 'B'
Attributes: (12/16)
    resolution:              10
    area:                    Area ID: 23KRR\nDescription: On-the-fly area\nPr...
    start_time:              2026-01-01 12:58:41.058082
    reader:                  msi_safe
    standard_name:           true_color
    end_time:                2026-01-01 12:58:41.058082
    ...                      ...
    wavelength:              None
    name:                    true_color
    _satpy_id:               DataID(name='true_color', resolution=10)
    prerequisites:           [DataQuery(name='B04', modifiers=('effective_sol...
    optional_prerequisites:  []
    mode:                    RGB

In [31]:
msi_corrected_scn.load(['satellite_azimuth_angle', 'satellite_zenith_angle',
                        'solar_azimuth_angle', 'solar_zenith_angle'])

# msi does not hvae sunz_corrected ... instead they call 'effective_solar_pathlength_corrected' in their composites generation
# see https://satpy.readthedocs.io/en/latest/api/satpy.modifiers.geometry.html for details and research paper

msi_corrected_scn.load(['B01', 'B02', 'B04', 'B05', 'B06', 'B07',
                        'B08', 'B09', 'B10', 'B11', 'B12', 'B8A'],
                       modifiers=['effective_solar_pathlength_corrected'])

msi_corrected_scn.load(['B03'], modifiers=['effective_solar_pathlength_corrected', 'rayleigh_corrected'])

msi_corrected_scn.load(['true_color'])

resampled_corrected_msi = msi_corrected_scn.resample(resampler='native')

print(resampled_corrected_msi['B01'].attrs['modifiers'])
print()
print(resampled_corrected_msi['B03'].attrs['modifiers'])
print()
print(resampled_corrected_msi['true_color'].prerequisites)



NameError: name 'msi_corrected_scn' is not defined

In [ ]:
for band in ['B01', 'B02', 'B04', 'B05', 'B06', 'B07',
             'B08', 'B09', 'B10', 'B11', 'B12', 'B8A']:
    print(band)
    print(msi_corrected_scn[band].attrs['calibration'])

In [141]:
test_dir = DATA_DIR + '/sars_p1_data/sentinel2b-msi'

In [142]:
filez = find_files_and_readers(base_dir=test_dir,
                               start_time=datetime(2025, 7, 25, 4, 0, 0),
                               end_time=datetime(2025, 7, 25, 8, 0, 0),
                               reader='msi_safe',)
print(filez)

{'msi_safe': ['/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20250725T103629_N0511_R008_T31TFL_20250725T123454.SAFE/GRANULE/L1C_T31TFL_A043792_20250725T103823/MTD_TL.xml', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE/L1C_T44TMK_A037454_20240508T053113/MTD_TL.xml', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNQ_20240508T072346.SAFE/GRANULE/L1C_T44TNQ_A037454_20240508T053113/MTD_TL.xml', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNP_20240508T072346.SAFE/GRANULE/L1C_T44TNP_A037454_20240508T053113/MTD_TL.xml', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TML_20240508T072346.SAFE/GRANULE/L1C_T44TML_A037454_20240508T053113/MTD_TL.xml', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20250502T092029_N0511_R093_T33QXU_20250502T1147

In [ ]:
groups = defaultdict(list)

for f in filez['msi_safe']:

    safe_name = Path(f).parts[6]  # grabs SAFE directory name
    groups[safe_name].append(f)
msi_scenes = {}
for safe, safe_files in groups.items():
    msi_scenes[safe] = Scene(filenames=safe_files, reader='msi_safe')
    print(safe, len(safe_files))

In [ ]:
msi_scenes

In [ ]:
Scene(filenames=filez)

## Using `process_satellite_data` function from satellite_data_processor.py

In [ ]:
data_dir = DATA_DIR
data_dir

In [ ]:
load_recipes = [(['1', '3', '4'], ['sunz_corrected', 'rayleigh_corrected']),
                (['true_color'], [])]

In [ ]:
# datasets = process_satellite_data(
#     data_dir=data_dir,
#     satpy_reader='modis_l1b',
#     start_time='20260101T1230',
#     end_time='20260101T1230',
#     satellite_name='Aqua',
#     satellite_instrument='MODIS',
#     load_recipes=load_recipes,
#     save_path=Path('~/Downloads').expanduser(),
#     correction_type='both'
# )

In [ ]:
file_list = find_files_and_readers(base_dir=DATA_DIR,
                                   start_time=datetime(2026, 1, 1, 12, 00),
                                   end_time=datetime(2026, 1, 1, 12, 00),
                                   reader='viirs_l1b')

scene = Scene(file_list)

In [ ]:
data_names = scene.available_dataset_names()
print(data_names)

In [ ]:
print(scene['B12'].standard_name)

In [ ]:
scene.load(data_names[:])

In [ ]:
for x in data_names:
    try:
        print(scene[x].standard_name, '   ---->    ', x)
    except:
        print(x)
        continue

In [ ]:
import os

def print_tree(start_path):
    for root, dirs, files in os.walk(start_path):
        level = root.replace(start_path, "").count(os.sep)
        indent = " " * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = " " * 4 * (level + 1)
        for f in files:
            print(f"{subindent}{f}")

In [ ]:
pth = str(Path('~/Downloads/sars_p1_data').expanduser())


In [ ]:
print_tree(pth)

In [ ]:
filez.items()

In [ ]:
pth + '/noaa20_viirs'

In [ ]:
os.listdir(pth+'/noaa20-viirs')

In [ ]:
viirstest = find_files_and_readers(base_dir = pth + '/noaa20-viirs',
                                   start_time=datetime(2024, 5, 8, 7, 0, 0),
                                   end_time=datetime(2024, 5, 8, 8, 0, 0),
                                   reader='viirs_l1b',)

In [131]:
viirstest

{'viirs_l1b': ['/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102IMG.A2024129.0736.021.2024129133614.nc',
  '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102IMG.A2024129.0730.021.2024129133608.nc',
  '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102MOD.A2024129.0736.021.2024129133614.nc',
  '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102MOD.A2024129.0730.021.2024129133608.nc']}

In [132]:
Scene(filez)

TypeError: '<' not supported between instances of 'method' and 'method'

In [136]:
filez['msi_safe']

['/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20250725T103629_N0511_R008_T31TFL_20250725T123454.SAFE/GRANULE/L1C_T31TFL_A043792_20250725T103823/MTD_TL.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE/L1C_T44TMK_A037454_20240508T053113/MTD_TL.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNQ_20240508T072346.SAFE/GRANULE/L1C_T44TNQ_A037454_20240508T053113/MTD_TL.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNP_20240508T072346.SAFE/GRANULE/L1C_T44TNP_A037454_20240508T053113/MTD_TL.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TML_20240508T072346.SAFE/GRANULE/L1C_T44TML_A037454_20240508T053113/MTD_TL.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20250502T092029_N0511_R093_T33QXU_20250502T114713.SAFE/

In [137]:
from satpy.readers import msi_safe

In [143]:
DATA_DIR = Path("~/Downloads/sars_p1_data").expanduser()
sentinel2b_dir = DATA_DIR / 'sentinel2b-msi'

In [153]:
sorted(sentinel2b_dir.glob('*'))[1:]

[PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TML_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMM_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMN_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNN_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNP_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TNQ_20240508T072346.SAFE'),
 PosixPath('/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_M

In [11]:
import numpy as np
import xarray as xr
from satpy import Scene, find_files_and_readers, DataQuery
from datetime import datetime
import os
from pathlib import Path
from satellite_data_processor import process_satellite_data
from sat_info import *
from collections import defaultdict


DATA_DIR = Path("~/Downloads/sars_p1_data").expanduser()
save_path = Path("~/Downloads/sars_p1_data/processed_output").expanduser()

# Create symlink to fix the naming issue ... you may need to change this depending on your system
pyspectral_dir = Path.home() / 'Library/Application Support/pyspectral'
source = pyspectral_dir / 'rsr_modis_EOS-Aqua.h5'
target = pyspectral_dir / 'rsr_modis_Aqua.h5'

if source.exists() and not target.exists():
    os.symlink(source, target)
    print(f"Created symlink: {target} -> {source}")
else:
    print(f"Source exists: {source.exists()}, Target exists: {target.exists()}")


terra_dir = DATA_DIR / 'terra-modis'
noaa20_dir = DATA_DIR / 'noaa20-viirs'
sentinel2b_dir = DATA_DIR / 'sentinel2b-msi'

granules = {
    "terra": [sorted(map(str, terra_dir.rglob('*2024129*'))), sorted(map(str, terra_dir.rglob('*2025122*'))), sorted(map(str, terra_dir.rglob('*2025206*')))],
    "noaa20": [sorted(map(str, noaa20_dir.rglob('*2024129*'))), sorted(map(str, noaa20_dir.rglob('*2025122*'))), sorted(map(str, noaa20_dir.rglob('*2025206*')))],
    "sentinel2b": [[f] for f in sorted(map(str, sentinel2b_dir.glob('*')))[1:]],
}


Source exists: True, Target exists: True


In [12]:
granules

{'terra': [['/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD021KM.A2024129.0500.061.2024129131921.hdf',
   '/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD03.A2024129.0500.061.2024129114512.hdf'],
  ['/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD021KM.A2025122.0830.061.2025122191112.hdf',
   '/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD03.A2025122.0500.061.2025122114934.hdf'],
  ['/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD021KM.A2025206.0935.061.2025206192851.hdf',
   '/Users/cwelch/Downloads/sars_p1_data/terra-modis/MOD03.A2025206.0935.061.2025206151823.hdf']],
 'noaa20': [['/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102IMG.A2024129.0730.021.2024129133608.nc',
   '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102IMG.A2024129.0736.021.2024129133614.nc',
   '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102MOD.A2024129.0730.021.2024129133608.nc',
   '/Users/cwelch/Downloads/sars_p1_data/noaa20-viirs/VJ102MOD.A2024129.0736.021.20241

In [66]:
test_list_sen = list(map(str, Path(granule_files[0]).rglob("*")))
test_list_sen

['/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/manifest.safe',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/INSPIRE.xml',
 '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/DATASTRIP',
 '/Users/cwelch/Downloads/sars_p1_data/se

In [67]:
Scene(filenames=test_list_sen, reader='msi_safe')

Don't know how to open the following files: {'/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE/L1C_T44TMK_A037454_20240508T053113/QI_DATA/MSK_CLASSI_B00.jp2', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/HTML/banner_2.png', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE/L1C_T44TMK_A037454_20240508T053113/QI_DATA/MSK_QUALIT_B09.jp2', '/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE/GRANULE/L1C_T44TMK_A037454_20240508T053113/QI_DATA/MSK_QUALIT_B03.jp2',

In [13]:
fname = Path(_gf).name
print(fname)
# if fname.startswith("S2"):
#     return fname.split('_')[2]  # Sentinel-2
#
# elif ".A" in fname:
#     parts = fname.split('.')
#     return parts[1] + "_" + parts[2]  # MODIS / VIIRS
#
# else:
#     raise ValueError("Unknown file format")

NameError: name '_gf' is not defined

In [89]:
for sat_name, data_info in satellite_data_info.items():
    for granule_files in granules[sat_name]:
        _gf = granule_files
        if sat_name == "sentinel2b":
            granule_files = find_files_and_readers(base_dir = granule_files[0], reader="msi_safe")

        scene = Scene(filenames=granule_files, reader=data_info['reader'])

        identifier = extract_identifier(_gf)

        print(scene.available_dataset_names())
        print()


['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9', 'height', 'landsea_mask', 'latitude', 'longitude', 'range', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'waterpresent']

['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9', 'height', 'landsea_mask', 'latitude', 'longitude', 'range', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'waterpresent']

['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '3

In [20]:
Path(granules['sentinel2b'][0][0]).name.split('_')[-2]

'T44TMK'

In [16]:
extract_identifier(granules['sentinel2b'][2])

'20240508T052649'

In [53]:
pth = str(Path('~/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE').expanduser())
pth
# find_files_and_readers(base_dir = pth, reader='msi_safe')

'/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE'

In [55]:
pth = str(Path('~/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE').expanduser())

Scene(filenames=[pth], reader='msi_safe')

No filenames found for reader: msi_safe
Don't know how to open the following files: {'/Users/cwelch/Downloads/sars_p1_data/sentinel2b-msi/S2B_MSIL1C_20240508T052649_N0510_R105_T44TMK_20240508T072346.SAFE'}


ValueError: No supported files found

In [ ]:
process_satellite_data()

In [54]:
import xarray as xr
data_dir = Path('~/Downloads/sars_p1_data/processed_output').expanduser()
for file in data_dir.glob('*.nc'):
    print(file)
    print(xr.open_dataset(file))
    print('##################################################################################################')
    print()

/Users/cwelch/Downloads/sars_p1_data/processed_output/noaa20_VIIRS_uncorrected_A2025206_1230.nc
ItemsView(<xarray.Dataset> Size: 1GB
Dimensions:                  (y: 3232, x: 3200)
Coordinates:
    longitude                (y, x) float32 41MB ...
    latitude                 (y, x) float32 41MB ...
Dimensions without coordinates: y, x
Data variables: (12/25)
    I01                      (y, x) float32 41MB ...
    I02                      (y, x) float32 41MB ...
    I03                      (y, x) float32 41MB ...
    I04                      (y, x) float32 41MB ...
    I05                      (y, x) float32 41MB ...
    M01                      (y, x) float32 41MB ...
    ...                       ...
    M15                      (y, x) float32 41MB ...
    M16                      (y, x) float32 41MB ...
    satellite_azimuth_angle  (y, x) float32 41MB ...
    satellite_zenith_angle   (y, x) float32 41MB ...
    solar_azimuth_angle      (y, x) float32 41MB ...
    solar_zenith_angle

In [55]:
test = xr.open_dataset(file)

In [56]:
test

<xarray.Dataset> Size: 1GB
Dimensions:                  (y: 3232, x: 3200)
Coordinates:
    longitude                (y, x) float32 41MB ...
    latitude                 (y, x) float32 41MB ...
Dimensions without coordinates: y, x
Data variables: (12/25)
    I01                      (y, x) float32 41MB ...
    I02                      (y, x) float32 41MB ...
    I03                      (y, x) float32 41MB ...
    I04                      (y, x) float32 41MB ...
    I05                      (y, x) float32 41MB ...
    M01                      (y, x) float32 41MB ...
    ...                       ...
    M15                      (y, x) float32 41MB ...
    M16                      (y, x) float32 41MB ...
    satellite_azimuth_angle  (y, x) float32 41MB ...
    satellite_zenith_angle   (y, x) float32 41MB ...
    solar_azimuth_angle      (y, x) float32 41MB ...
    solar_zenith_angle       (y, x) float32 41MB ...
Attributes:
    history:      Created by pytroll/satpy on 2026-02-17 03:16:20.553410+00:00
    Conventions:  CF-1.7